In [ ]:
# Install the necessary library to download files from Google Drive
!pip install gdown

# Download the shared folder
!gdown --folder https://drive.google.com/drive/folders/1MRPXz79Lu_JGLlJ21MDfML44dKN9R08l

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

In [ ]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
# CELL 2 (New Cell)
def check_tensor(tensor, name="Tensor"):
    """Checks a tensor for NaN or Inf values."""
    if torch.isnan(tensor).any() or torch.isinf(tensor).any():
        print(f"🔴 ALERT: '{name}' contains NaN or Inf values!")
        return True
    else:
        # This print statement is optional, you can comment it out if the log is too noisy
        print(f"✅ OK: '{name}' is clean.")
        return False

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv, global_mean_pool
# Re-import NeighborLoader for mini-batch training
from torch_geometric.loader import NeighborLoader
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, f1_score
from sklearn.utils.class_weight import compute_class_weight
import warnings
import pickle
import os
import json
warnings.filterwarnings('ignore')

# CHECKPOINT SYSTEM
CHECKPOINT_DIR = 'Ransomware_Checkpoints'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

def save_checkpoint(checkpoint_name, data, message="Checkpoint saved"):
    """Save checkpoint data"""
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f"{checkpoint_name}.pkl")
    with open(checkpoint_path, 'wb') as f:
        pickle.dump(data, f)
    print(f"✅ {message}: {checkpoint_path}")

def load_checkpoint(checkpoint_name):
    """Load checkpoint data"""
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f"{checkpoint_name}.pkl")
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'rb') as f:
            return pickle.load(f)
    return None

def checkpoint_exists(checkpoint_name):
    """Check if checkpoint exists"""
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f"{checkpoint_name}.pkl")
    return os.path.exists(checkpoint_path)

# Updated title for clarity
print("🚀 IMPROVED GNN RANSOMWARE DETECTION MODEL WITH NEIGHBORLOADER")
print("="*70)

# STEP 1: Check for data loading checkpoint
if checkpoint_exists('data_loaded'):
    print("[RESUME] Loading data from checkpoint...")
    checkpoint_data = load_checkpoint('data_loaded')
    wallets_df = checkpoint_data['wallets_df']
    edges_df = checkpoint_data['edges_df']
    print(f"[SUCCESS] Resumed with {len(wallets_df)} wallet records and {len(edges_df)} edges")
else:
    print("[INFO] Loading wallet features and classes...")
    wallets_df = pd.read_csv('Elliptic++ Dataset/wallets_features_classes_combined.csv')
    print(f"[SUCCESS] Loaded {len(wallets_df)} wallet records")

    print("[INFO] Loading address-to-address edges...")
    edges_df = pd.read_csv('Elliptic++ Dataset/AddrAddr_edgelist.csv')
    print(f"[SUCCESS] Loaded {len(edges_df)} edges")

    # Save checkpoint
    save_checkpoint('data_loaded', {
        'wallets_df': wallets_df,
        'edges_df': edges_df
    }, "Data loading checkpoint saved")

# STEP 2: Check for feature engineering checkpoint
def create_enhanced_pattern_features(df):
    """Create the EXACT same enhanced features as XGBoost model"""

    # Fill missing values first (same as XGBoost)
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df[numeric_columns] = df[numeric_columns].fillna(0)

    # 1. Partner Transaction Ratio (connectivity)
    df['partner_transaction_ratio'] = (
        df.get('transacted_w_address_total', 0) /
        (df.get('total_txs', 1) + 1e-8)
    )

    # 2. Activity Density (txs per block)
    df['activity_density'] = (
        df.get('total_txs', 0) /
        (df.get('lifetime_in_blocks', 1) + 1e-8)
    )

    # 3. Transaction Size Variance (volatility)
    df['transaction_size_variance'] = (
        df.get('btc_transacted_max', 0) - df.get('btc_transacted_min', 0)
    ) / (df.get('btc_transacted_mean', 1) + 1e-8)

    # 4. Flow Imbalance (money laundering indicator)
    df['flow_imbalance'] = (
        (df.get('btc_sent_total', 0) - df.get('btc_received_total', 0)) /
        (df.get('btc_transacted_total', 1) + 1e-8)
    )

    # 5. Temporal Spread (time pattern)
    df['temporal_spread'] = (
        df.get('last_block_appeared_in', 0) - df.get('first_block_appeared_in', 0)
    ) / (df.get('num_timesteps_appeared_in', 1) + 1e-8)

    # 6. Fee Percentile (urgency indicator)
    df['fee_percentile'] = (
        df.get('fees_total', 0) /
        (df.get('btc_transacted_total', 1) + 1e-8)
    )

    # 7. Interaction Intensity (network centrality)
    df['interaction_intensity'] = (
        df.get('num_addr_transacted_multiple', 0) /
        (df.get('transacted_w_address_total', 1) + 1e-8)
    )

    # 8. Value Per Transaction (transaction size)
    df['value_per_transaction'] = (
        df.get('btc_transacted_total', 0) /
        (df.get('total_txs', 1) + 1e-8)
    )

    # 9. RANSOMWARE-SPECIFIC: Burst Activity (rapid txs)
    df['burst_activity'] = (
        df.get('total_txs', 0) * df.get('activity_density', 0)
    )

    # 10. RANSOMWARE-SPECIFIC: Mixing Intensity (obfuscation)
    df['mixing_intensity'] = (
        df.get('partner_transaction_ratio', 0) * df.get('interaction_intensity', 0)
    )

    return df

if checkpoint_exists('features_engineered'):
    print("[RESUME] Loading feature engineering from checkpoint...")
    checkpoint_data = load_checkpoint('features_engineered')
    wallets_df = checkpoint_data['wallets_df']
    print(f"[SUCCESS] Resumed with {len(wallets_df.columns)} columns")
else:
    print("[INFO] Creating enhanced features...")
    wallets_df = create_enhanced_pattern_features(wallets_df)
    print(f"[SUCCESS] Enhanced features created. Now have {len(wallets_df.columns)} columns")

    # Save checkpoint
    save_checkpoint('features_engineered', {
        'wallets_df': wallets_df
    }, "Feature engineering checkpoint saved")

# STEP 3: Check for data cleaning checkpoint
if checkpoint_exists('data_cleaned'):
    print("[RESUME] Loading cleaned data from checkpoint...")
    checkpoint_data = load_checkpoint('data_cleaned')
    wallets_clean = checkpoint_data['wallets_clean']
    class_weight_dict = checkpoint_data['class_weight_dict']
    print(f"[SUCCESS] Resumed with {len(wallets_clean)} cleaned addresses")
    print(f"[INFO] Class distribution: {wallets_clean['class'].value_counts().to_dict()}")
    print(f"[INFO] Class weights: {class_weight_dict}")
else:
    print("[INFO] Cleaning data with same logic as XGBoost...")

    # Keep only labeled addresses (1=Illicit, 2=Licit)
    wallets_clean = wallets_df[wallets_df['class'].isin([1, 2])].copy()
    print(f"[INFO] Labeled addresses: {len(wallets_clean)}")

    # Remap classes: 1->1 (Illicit), 2->0 (Licit) - SAME AS XGBOOST
    wallets_clean['class'] = wallets_clean['class'].map({1: 1, 2: 0})
    print(f"[INFO] Class distribution: {wallets_clean['class'].value_counts().to_dict()}")

    # Calculate class weights (same as XGBoost)
    class_weights = compute_class_weight('balanced', classes=np.unique(wallets_clean['class']), y=wallets_clean['class'])
    class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
    print(f"[INFO] Class weights: {class_weight_dict}")

    # Save checkpoint
    save_checkpoint('data_cleaned', {
        'wallets_clean': wallets_clean,
        'class_weight_dict': class_weight_dict
    }, "Data cleaning checkpoint saved")

# STEP 4: Check for graph construction checkpoint
if checkpoint_exists('graph_built'):
    print("[RESUME] Loading graph from checkpoint...")
    checkpoint_data = load_checkpoint('graph_built')
    data = checkpoint_data['data']
    addr_to_idx = checkpoint_data['addr_to_idx']
    feature_cols = checkpoint_data['feature_cols']
    scaler = checkpoint_data['scaler']
    X_scaled = checkpoint_data['X_scaled']
    print(f"[SUCCESS] Resumed with graph: {data.x.shape[0]} nodes, {data.edge_index.shape[1]} edges")
else:
    print("[INFO] Building improved graph structure...")
    print("This is the slow part - creating address mappings and features...")

    # Create address to index mapping
    unique_addresses = wallets_clean['address'].unique()
    addr_to_idx = {addr: idx for idx, addr in enumerate(unique_addresses)}

    # Prepare features and labels
    exclude_cols = ['address', 'Time step', 'class']
    feature_cols = [col for col in wallets_clean.columns if col not in exclude_cols]

    features_list = []
    labels_list = []
    address_list = []

    print(f"[INFO] Processing {len(unique_addresses)} unique addresses...")
    for i, addr in enumerate(unique_addresses):
        if i % 10000 == 0:
            print(f"  Progress: {i}/{len(unique_addresses)} ({i/len(unique_addresses)*100:.1f}%)")

        addr_data = wallets_clean[wallets_clean['address'] == addr].iloc[0]
        features_list.append(addr_data[feature_cols].values)
        labels_list.append(addr_data['class'])
        address_list.append(addr)

    X = np.array(features_list)
    y = np.array(labels_list)

    print(f"[INFO] Feature matrix shape: {X.shape}")
    print(f"[INFO] Labels shape: {y.shape}")

    # Create edge list with weights
    print("[INFO] Creating weighted edges...")
    edge_list = []
    edge_weights = []

    print(f"[INFO] Processing {len(edges_df)} edges...")
    for i, (_, row) in enumerate(edges_df.iterrows()):
        if i % 50000 == 0:
            print(f"  Edge progress: {i}/{len(edges_df)} ({i/len(edges_df)*100:.1f}%)")

        input_addr = row['input_address']
        output_addr = row['output_address']

        if input_addr in addr_to_idx and output_addr in addr_to_idx:
            input_idx = addr_to_idx[input_addr]
            output_idx = addr_to_idx[output_addr]

            # Add both directions for undirected graph
            edge_list.extend([[input_idx, output_idx], [output_idx, input_idx]])

            # Use transaction amount as edge weight if available
            weight = row.get('value', 1.0) if 'value' in row else 1.0
            edge_weights.extend([weight, weight])

    edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
    edge_weights = torch.tensor(edge_weights, dtype=torch.float)

    print(f"[INFO] Graph edges: {edge_index.shape[1]} edges (bidirectional)")

    # Scale features
    print("[INFO] Scaling features with RobustScaler...")
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X)

    # Convert to PyTorch tensors
    x = torch.tensor(X_scaled, dtype=torch.float)
    y = torch.tensor(y, dtype=torch.long)

    print(f"[INFO] Final graph structure:")
    print(f"  - Nodes: {x.shape[0]}")
    print(f"  - Node features: {x.shape[1]}")
    f"  - Edges: {edge_index.shape[1]}"
    print(f"  - Classes: Licit={sum(y==0)}, Illicit={sum(y==1)}")

    # Create PyTorch Geometric data object
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_weights, y=y)

    # Save checkpoint
    save_checkpoint('graph_built', {
        'data': data,
        'addr_to_idx': addr_to_idx,
        'feature_cols': feature_cols,
        'scaler': scaler,
        'X_scaled': X_scaled
    }, "Graph construction checkpoint saved")

# STEP 5: Model Definition (Updated for batch processing and torch.nn.BatchNorm1d)
class ImprovedGNN(nn.Module):
    def __init__(self, num_features, hidden_dim=128, num_classes=2, dropout=0.3):
        super(ImprovedGNN, self).__init__()

        # Graph attention layers for better feature learning
        self.gat1 = GATConv(num_features, hidden_dim, heads=4, dropout=dropout)
        # Use standard torch.nn.BatchNorm1d
        self.bn1 = torch.nn.BatchNorm1d(hidden_dim * 4)

        self.gat2 = GATConv(hidden_dim * 4, hidden_dim, heads=4, dropout=dropout)
        # Use standard torch.nn.BatchNorm1d
        self.bn2 = torch.nn.BatchNorm1d(hidden_dim * 4)

        self.gat3 = GATConv(hidden_dim * 4, hidden_dim // 2, heads=2, dropout=dropout)
        # Use standard torch.nn.BatchNorm1d
        self.bn3 = torch.nn.BatchNorm1d(hidden_dim) # output shape after gat3 is hidden_dim // 2 * 2 = hidden_dim

        # Feature-only path (like XGBoost)
        self.feature_path = nn.Sequential(
            nn.Linear(num_features, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.Dropout(dropout)
        )

        # Combined classifier
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim + hidden_dim // 2, hidden_dim // 2),  # Graph + Feature paths
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 32),
            nn.ReLU(),
            nn.Dropout(dropout // 2),
            nn.Linear(32, num_classes)
        )

        self.dropout = dropout

    def forward(self, x, edge_index, batch=None):
        # Graph path
        h1 = F.elu(self.gat1(x, edge_index))
        h1 = self.bn1(h1)
        h1 = F.dropout(h1, p=self.dropout, training=self.training)

        h2 = F.elu(self.gat2(h1, edge_index))
        h2 = self.bn2(h2)
        h2 = F.dropout(h2, p=self.dropout, training=self.training)

        h3 = F.elu(self.gat3(h2, edge_index))
        h3 = self.bn3(h3)
        graph_features = F.dropout(h3, p=self.dropout, training=self.training)

        # Feature-only path (mimics XGBoost)
        feature_only = self.feature_path(x)

        # Combine both paths
        combined = torch.cat([graph_features, feature_only], dim=1)

        # Classification
        out = self.classifier(combined)

        return out

# STEP 6: Training setup with NeighborLoader - FIXED CUDA DETECTION
def save_training_checkpoint(epoch, model, optimizer, scheduler, best_f1, patience_counter, train_losses, test_f1_scores):
    """Save training checkpoint"""
    training_state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
        'best_f1': best_f1,
        'patience_counter': patience_counter,
        'train_losses': train_losses,
        'test_f1_scores': test_f1_scores
    }
    torch.save(training_state, os.path.join(CHECKPOINT_DIR, 'training_checkpoint.pth'))
    print(f"💾 Training checkpoint saved at epoch {epoch}")

# FORCE CUDA DETECTION - Don't load device from checkpoint
print("[INFO] Forcing CUDA device detection...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"[INFO] CUDA Available: {torch.cuda.is_available()}")
print(f"[INFO] Device: {device}")

if torch.cuda.is_available():
    print(f"[INFO] CUDA Device: {torch.cuda.get_device_name(0)}")
    print(f"[INFO] CUDA Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Check for training setup checkpoint - but don't load device from it
if checkpoint_exists('training_setup'):
    print("[RESUME] Loading training setup from checkpoint...")
    checkpoint_data = load_checkpoint('training_setup')
    train_mask = checkpoint_data['train_mask']
    test_mask = checkpoint_data['test_mask']
    # DON'T LOAD DEVICE FROM CHECKPOINT - use current detection
    print(f"[SUCCESS] Resumed training setup")
    print(f"[INFO] Train samples: {train_mask.sum()}")
    print(f"[INFO] Test samples: {test_mask.sum()}")
    print(f"[INFO] Device: {device}")  # Use current device
else:
    print("[INFO] Creating stratified train-test split...")
    train_idx, test_idx = train_test_split(
        range(len(data.y)), test_size=0.2, random_state=42,
        stratify=data.y.numpy()
    )

    train_mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    test_mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    train_mask[train_idx] = True
    test_mask[test_idx] = True

    print(f"[INFO] Train samples: {train_mask.sum()}")
    print(f"[INFO] Test samples: {test_mask.sum()}")
    print(f"[INFO] Device: {device}")

    # Save checkpoint - but device will be re-detected next time
    save_checkpoint('training_setup', {
        'train_mask': train_mask,
        'test_mask': test_mask,
        'device': device  # This will be ignored on reload
    }, "Training setup checkpoint saved")

# Initialize model and training components
print(f"[INFO] Initializing model on device: {device}")
model = ImprovedGNN(num_features=data.x.shape[1], hidden_dim=128, dropout=0.3).to(device)

# IMPORTANT: Move data to device for NeighborLoader
print("[INFO] Moving graph data to device...")
data = data.to(device)

# Weighted loss for imbalanced classes
class_weights_tensor = torch.tensor([class_weight_dict[0], class_weight_dict[1]], dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=20)

# STEP 7: CREATE NEIGHBORLOADER FOR MEMORY-EFFICIENT TRAINING
print("[INFO] Creating NeighborLoader for memory-efficient training...")

# Create training and test loaders with neighbor sampling
train_loader = NeighborLoader(
    data,
    num_neighbors=[15, 10, 5],  # Sample 15 neighbors for layer 1, 10 for layer 2, 5 for layer 3
    batch_size=1024,  # Process 1024 nodes at a time
    input_nodes=train_mask,
    shuffle=True,
    num_workers= 4  # Set to 0 for Colab compatibility
)

test_loader = NeighborLoader(
    data,
    num_neighbors=[15, 10, 5],
    batch_size=1024,
    input_nodes=test_mask,
    shuffle=False,
    num_workers= 4
)

print(f"[INFO] Training batches: {len(train_loader)}")
print(f"[INFO] Test batches: {len(test_loader)}")

# Check for training resume
training_checkpoint_path = os.path.join(CHECKPOINT_DIR, 'training_checkpoint.pth')
start_epoch = 0
best_f1 = 0
patience_counter = 0
train_losses = []
test_f1_scores = []

if os.path.exists(training_checkpoint_path):
    print("[RESUME] Loading training checkpoint...")
    checkpoint = torch.load(training_checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if checkpoint['scheduler_state_dict']:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    best_f1 = checkpoint['best_f1']
    patience_counter = checkpoint['patience_counter']
    train_losses = checkpoint['train_losses']
    test_f1_scores = checkpoint['test_f1_scores']
    print(f"[SUCCESS] Resumed training from epoch {start_epoch}, best F1: {best_f1:.4f}")

print(f"[INFO] Model parameters: {sum(p.numel() for p in model.parameters())}")
print(f"[INFO] Model device: {next(model.parameters()).device}")
print(f"[INFO] Data device: {data.x.device}")

def main():

    global start_epoch, best_f1, patience_counter
    
    # STEP 8: Training loop with NeighborLoader (MEMORY EFFICIENT)
    print("\n" + "="*70)
    print("🚀 TRAINING WITH NEIGHBORLOADER - MEMORY EFFICIENT ON CUDA")
    print("="*70)

    patience = 50
    max_epochs = 500

    for epoch in range(start_epoch, max_epochs):
        # Training with mini-batches
        model.train()
        total_loss = 0
        num_batches = 0

        for batch in train_loader:
            # Batch should already be on the correct device since data is on device
            # But let's be explicit
            batch = batch.to(device)
            optimizer.zero_grad()

            # Forward pass on batch
            out = model(batch.x, batch.edge_index)
            loss = criterion(out, batch.y)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()
            num_batches += 1

        avg_loss = total_loss / num_batches
        train_losses.append(avg_loss)

        # Evaluation every 10 epochs
        if epoch % 10 == 0:
            model.eval()
            all_preds = []
            all_labels = []
            all_probs = []

            with torch.no_grad():
                for batch in test_loader:
                    batch = batch.to(device)
                    out = model(batch.x, batch.edge_index)
                    pred_proba = F.softmax(out, dim=1)[:, 1]
                    pred = out.argmax(dim=1)

                    all_preds.extend(pred.cpu().numpy())
                    all_labels.extend(batch.y.cpu().numpy())
                    all_probs.extend(pred_proba.cpu().numpy())

            # Convert to numpy arrays
            y_test_np = np.array(all_labels)
            y_pred_np = np.array(all_preds)
            y_pred_proba_np = np.array(all_probs)

            # Find optimal threshold
            if len(np.unique(y_test_np)) > 1:
                precision, recall, thresholds = precision_recall_curve(y_test_np, y_pred_proba_np)
                f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
                best_threshold_idx = np.argmax(f1_scores)
                best_threshold = thresholds[best_threshold_idx] if len(thresholds) > 0 else 0.5

                y_pred_threshold = (y_pred_proba_np >= best_threshold).astype(int)
                test_f1 = f1_score(y_test_np, y_pred_threshold)
            else:
                test_f1 = 0
                best_threshold = 0.5

            test_f1_scores.append(test_f1)

            # Calculate training accuracy (sample from training loader)
            model.eval()
            train_preds_agg = []
            train_labels_agg = []
            train_batches_checked = 0

            with torch.no_grad():
                for batch in train_loader:
                    if train_batches_checked >= 5:  # Sample first 5 batches for speed
                        break
                    batch = batch.to(device)
                    out = model(batch.x, batch.edge_index)
                    pred = out.argmax(dim=1)
                    train_preds_agg.extend(pred.cpu().numpy())
                    train_labels_agg.extend(batch.y.cpu().numpy())
                    train_batches_checked += 1

            train_acc = np.mean(np.array(train_preds_agg) == np.array(train_labels_agg))
            test_acc = np.mean(y_pred_np == y_test_np)

            print(f'Epoch {epoch:03d}, Loss: {avg_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, F1: {test_f1:.4f}, Threshold: {best_threshold:.3f}')

            # Early stopping based on F1 score
            if test_f1 > best_f1:
                best_f1 = test_f1
                patience_counter = 0
                # Save best model
                torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, 'best_gnn_model.pth'))
            else:
                patience_counter += 1

            # Save training checkpoint every 20 epochs
            if epoch % 20 == 0:
                save_training_checkpoint(epoch, model, optimizer, scheduler, best_f1, patience_counter, train_losses, test_f1_scores)

            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

            scheduler.step(test_f1)

    print(f"\n✅ TRAINING COMPLETED ON: {device}")
    print(f"📊 Final device check - Model: {next(model.parameters()).device}")

    # Final checkpoint save
    save_training_checkpoint(epoch, model, optimizer, scheduler, best_f1, patience_counter, train_losses, test_f1_scores)

    # Load best model for final evaluation
    best_model_path = os.path.join(CHECKPOINT_DIR, 'best_gnn_model.pth')
    if os.path.exists(best_model_path):
        model.load_state_dict(torch.load(best_model_path, map_location=device))

    # STEP 9: Final evaluation with NeighborLoader (using aggregated results from test loader)
    print("\n" + "="*70)
    print("📊 FINAL EVALUATION WITH NEIGHBORLOADER (Aggregated Test Batches)")
    print("="*70)

    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            out = model(batch.x, batch.edge_index)
            pred_proba = F.softmax(out, dim=1)[:, 1]
            pred = out.argmax(dim=1)

            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(batch.y.cpu().numpy())
            all_probs.extend(pred_proba.cpu().numpy())

    # Convert to numpy arrays
    y_test = np.array(all_labels)
    y_pred_proba_test = np.array(all_probs)

    # Find optimal threshold
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba_test)
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
    best_threshold_idx = np.argmax(f1_scores)
    optimal_threshold = thresholds[best_threshold_idx] if len(thresholds) > 0 else 0.5 # Handle case with no positive predictions

    y_pred_test = (y_pred_proba_test >= optimal_threshold).astype(int)

    # Calculate metrics
    # Note: Accuracy is calculated on aggregated batch predictions, not the true overall test set.
    test_acc = np.mean(np.array(all_preds) == y_test)
    # AUC and F1 are correctly calculated on aggregated test predictions and probabilities
    auc_score = roc_auc_score(y_test, y_pred_proba_test) if len(np.unique(y_test)) > 1 else 0.0 # Handle case with no positive labels in test set
    final_f1 = f1_score(y_test, y_pred_test) if len(np.unique(y_test)) > 1 else 0.0


    print(f"Test Accuracy (Aggregated): {test_acc:.4f}")
    print(f"AUC Score (Aggregated): {auc_score:.4f}")
    print(f"F1 Score (Aggregated): {final_f1:.4f}")
    print(f"Optimal Threshold: {optimal_threshold:.3f}")

    print("\nClassification Report (Aggregated Test Batches):")
    # Ensure target names match unique labels present in y_test
    target_names = ['Licit', 'Illicit']
    report = classification_report(y_test, y_pred_test, target_names=target_names, zero_division=0)
    print(report)


    print("\nConfusion Matrix (Aggregated Test Batches):")
    cm = confusion_matrix(y_test, y_pred_test)
    print("   Predicted:  Licit  Illicit")
    print(f"   Licit:      {cm[0,0]:5d}     {cm[0,1]:4d}")
    print(f"   Illicit:      {cm[1,0]:3d}     {cm[1,1]:4d}")


    # STEP 10: Save the final model
    print("\n" + "="*70)
    print("💾 SAVING FINAL MODEL WITH NEIGHBORLOADER SUPPORT")
    print("="*70)

    model_data = {
        'model_state_dict': model.state_dict(),
        'scaler': scaler,
        'addr_to_idx': addr_to_idx,
        'feature_cols': feature_cols,
        'optimal_threshold': optimal_threshold,
        'class_weights': class_weight_dict,
        'model_config': {
            'num_features': data.x.shape[1],
            'hidden_dim': 128,
            'num_classes': 2,
            'dropout': 0.3
        },
        'training_history': {
            'train_losses': train_losses,
            'test_f1_scores': test_f1_scores,
            'final_metrics': {
                'test_accuracy': test_acc, # Note: Aggregated accuracy
                'auc_score': auc_score,
                'f1_score': final_f1,
                'optimal_threshold': optimal_threshold
            }
        }
    }

    torch.save(model_data, 'improved_gnn_ransomware_model_neighborloader.pth')
    print("[SUCCESS] Final NeighborLoader model saved!")

if __name__ == '__main__':
    # Ensure num_workers is set to > 0 in STEP 7
    # For example: num_workers=4
    main()
# Clean up checkpoints (optional)
print("\n🧹 CLEANUP OPTIONS:")
print("To clean up checkpoints and save space, run:")
print("  import shutil; shutil.rmtree('/content/checkpoints')")

print(f"\n🎯 NEIGHBORLOADER TRAINING ADOPTED:")
print(f"  ✅ Re-enabled NeighborLoader for memory efficiency")
print(f"  ✅ Training and evaluation using mini-batches")
print(f"  ✅ Requires torch-sparse (installed)")
print(f"  ⚠️ Evaluation metrics are aggregated over test batches, not the entire test set at once.")

print(f"\n📊 Training Attempted:")
print(f"  - Training with NeighborLoader should now proceed if torch-sparse is correctly installed.")

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv, global_mean_pool
# Re-import NeighborLoader for mini-batch training
from torch_geometric.loader import NeighborLoader
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, f1_score
from sklearn.utils.class_weight import compute_class_weight
import warnings
import pickle
import os
import json
warnings.filterwarnings('ignore')


# CHECKPOINT SYSTEM
CHECKPOINT_DIR = 'Ransomware_Checkpoints'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

def save_checkpoint(checkpoint_name, data, message="Checkpoint saved"):
    """Save checkpoint data"""
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f"{checkpoint_name}.pkl")
    with open(checkpoint_path, 'wb') as f:
        pickle.dump(data, f)
    print(f"✅ {message}: {checkpoint_path}")

def load_checkpoint(checkpoint_name):
    """Load checkpoint data"""
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f"{checkpoint_name}.pkl")
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'rb') as f:
            return pickle.load(f)
    return None

def checkpoint_exists(checkpoint_name):
    """Check if checkpoint exists"""
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f"{checkpoint_name}.pkl")
    return os.path.exists(checkpoint_path)

# Updated title for clarity
print("🚀 IMPROVED GNN RANSOMWARE DETECTION MODEL WITH NEIGHBORLOADER")
print("="*70)

# STEP 1: Check for data loading checkpoint
if checkpoint_exists('data_loaded'):
    print("[RESUME] Loading data from checkpoint...")
    checkpoint_data = load_checkpoint('data_loaded')
    wallets_df = checkpoint_data['wallets_df']
    edges_df = checkpoint_data['edges_df']
    print(f"[SUCCESS] Resumed with {len(wallets_df)} wallet records and {len(edges_df)} edges")
else:
    print("[INFO] Loading wallet features and classes...")
    wallets_df = pd.read_csv('Elliptic++ Dataset/wallets_features_classes_combined.csv')
    print(f"[SUCCESS] Loaded {len(wallets_df)} wallet records")

    print("[INFO] Loading address-to-address edges...")
    edges_df = pd.read_csv('Elliptic++ Dataset/AddrAddr_edgelist.csv')
    print(f"[SUCCESS] Loaded {len(edges_df)} edges")

    # Save checkpoint
    save_checkpoint('data_loaded', {
        'wallets_df': wallets_df,
        'edges_df': edges_df
    }, "Data loading checkpoint saved")

# STEP 2: Check for feature engineering checkpoint
def create_enhanced_pattern_features(df):
    """Create the EXACT same enhanced features as XGBoost model"""

    # Fill missing values first (same as XGBoost)
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df[numeric_columns] = df[numeric_columns].fillna(0)

    # 1. Partner Transaction Ratio (connectivity)
    df['partner_transaction_ratio'] = (
        df.get('transacted_w_address_total', 0) /
        (df.get('total_txs', 1) + 1e-8)
    )

    # 2. Activity Density (txs per block)
    df['activity_density'] = (
        df.get('total_txs', 0) /
        (df.get('lifetime_in_blocks', 1) + 1e-8)
    )

    # 3. Transaction Size Variance (volatility)
    df['transaction_size_variance'] = (
        df.get('btc_transacted_max', 0) - df.get('btc_transacted_min', 0)
    ) / (df.get('btc_transacted_mean', 1) + 1e-8)

    # 4. Flow Imbalance (money laundering indicator)
    df['flow_imbalance'] = (
        (df.get('btc_sent_total', 0) - df.get('btc_received_total', 0)) /
        (df.get('btc_transacted_total', 1) + 1e-8)
    )

    # 5. Temporal Spread (time pattern)
    df['temporal_spread'] = (
        df.get('last_block_appeared_in', 0) - df.get('first_block_appeared_in', 0)
    ) / (df.get('num_timesteps_appeared_in', 1) + 1e-8)

    # 6. Fee Percentile (urgency indicator)
    df['fee_percentile'] = (
        df.get('fees_total', 0) /
        (df.get('btc_transacted_total', 1) + 1e-8)
    )

    # 7. Interaction Intensity (network centrality)
    df['interaction_intensity'] = (
        df.get('num_addr_transacted_multiple', 0) /
        (df.get('transacted_w_address_total', 1) + 1e-8)
    )

    # 8. Value Per Transaction (transaction size)
    df['value_per_transaction'] = (
        df.get('btc_transacted_total', 0) /
        (df.get('total_txs', 1) + 1e-8)
    )

    # 9. RANSOMWARE-SPECIFIC: Burst Activity (rapid txs)
    df['burst_activity'] = (
        df.get('total_txs', 0) * df.get('activity_density', 0)
    )

    # 10. RANSOMWARE-SPECIFIC: Mixing Intensity (obfuscation)
    df['mixing_intensity'] = (
        df.get('partner_transaction_ratio', 0) * df.get('interaction_intensity', 0)
    )

    return df

if checkpoint_exists('features_engineered'):
    print("[RESUME] Loading feature engineering from checkpoint...")
    checkpoint_data = load_checkpoint('features_engineered')
    wallets_df = checkpoint_data['wallets_df']
    print(f"[SUCCESS] Resumed with {len(wallets_df.columns)} columns")
else:
    print("[INFO] Creating enhanced features...")
    wallets_df = create_enhanced_pattern_features(wallets_df)
    print(f"[SUCCESS] Enhanced features created. Now have {len(wallets_df.columns)} columns")

    # Save checkpoint
    save_checkpoint('features_engineered', {
        'wallets_df': wallets_df
    }, "Feature engineering checkpoint saved")

# STEP 3: Check for data cleaning checkpoint
if checkpoint_exists('data_cleaned'):
    print("[RESUME] Loading cleaned data from checkpoint...")
    checkpoint_data = load_checkpoint('data_cleaned')
    wallets_clean = checkpoint_data['wallets_clean']
    class_weight_dict = checkpoint_data['class_weight_dict']
    print(f"[SUCCESS] Resumed with {len(wallets_clean)} cleaned addresses")
    print(f"[INFO] Class distribution: {wallets_clean['class'].value_counts().to_dict()}")
    print(f"[INFO] Class weights: {class_weight_dict}")
else:
    print("[INFO] Cleaning data with same logic as XGBoost...")

    # Keep only labeled addresses (1=Illicit, 2=Licit)
    wallets_clean = wallets_df[wallets_df['class'].isin([1, 2])].copy()
    print(f"[INFO] Labeled addresses: {len(wallets_clean)}")

    # Remap classes: 1->1 (Illicit), 2->0 (Licit) - SAME AS XGBOOST
    wallets_clean['class'] = wallets_clean['class'].map({1: 1, 2: 0})
    print(f"[INFO] Class distribution: {wallets_clean['class'].value_counts().to_dict()}")

    # Calculate class weights (same as XGBoost)
    class_weights = compute_class_weight('balanced', classes=np.unique(wallets_clean['class']), y=wallets_clean['class'])
    class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
    print(f"[INFO] Class weights: {class_weight_dict}")

    # Save checkpoint
    save_checkpoint('data_cleaned', {
        'wallets_clean': wallets_clean,
        'class_weight_dict': class_weight_dict
    }, "Data cleaning checkpoint saved")

# STEP 4: Check for graph construction checkpoint
if checkpoint_exists('graph_built'):
    print("[RESUME] Loading graph from checkpoint...")
    checkpoint_data = load_checkpoint('graph_built')
    data = checkpoint_data['data']
    addr_to_idx = checkpoint_data['addr_to_idx']
    feature_cols = checkpoint_data['feature_cols']
    scaler = checkpoint_data['scaler']
    X_scaled = checkpoint_data['X_scaled']
    print(f"[SUCCESS] Resumed with graph: {data.x.shape[0]} nodes, {data.edge_index.shape[1]} edges")
else:
    print("[INFO] Building improved graph structure...")
    print("This is the slow part - creating address mappings and features...")

    # Create address to index mapping
    unique_addresses = wallets_clean['address'].unique()
    addr_to_idx = {addr: idx for idx, addr in enumerate(unique_addresses)}

    # Prepare features and labels
    exclude_cols = ['address', 'Time step', 'class']
    feature_cols = [col for col in wallets_clean.columns if col not in exclude_cols]

    features_list = []
    labels_list = []
    address_list = []

    print(f"[INFO] Processing {len(unique_addresses)} unique addresses...")
    for i, addr in enumerate(unique_addresses):
        if i % 10000 == 0:
            print(f"  Progress: {i}/{len(unique_addresses)} ({i/len(unique_addresses)*100:.1f}%)")

        addr_data = wallets_clean[wallets_clean['address'] == addr].iloc[0]
        features_list.append(addr_data[feature_cols].values)
        labels_list.append(addr_data['class'])
        address_list.append(addr)

    X = np.array(features_list)
    y = np.array(labels_list)

    print(f"[INFO] Feature matrix shape: {X.shape}")
    print(f"[INFO] Labels shape: {y.shape}")

    # Create edge list with weights
    print("[INFO] Creating weighted edges...")
    edge_list = []
    edge_weights = []

    print(f"[INFO] Processing {len(edges_df)} edges...")
    for i, (_, row) in enumerate(edges_df.iterrows()):
        if i % 50000 == 0:
            print(f"  Edge progress: {i}/{len(edges_df)} ({i/len(edges_df)*100:.1f}%)")

        input_addr = row['input_address']
        output_addr = row['output_address']

        if input_addr in addr_to_idx and output_addr in addr_to_idx:
            input_idx = addr_to_idx[input_addr]
            output_idx = addr_to_idx[output_addr]

            # Add both directions for undirected graph
            edge_list.extend([[input_idx, output_idx], [output_idx, input_idx]])

            # Use transaction amount as edge weight if available
            weight = row.get('value', 1.0) if 'value' in row else 1.0
            edge_weights.extend([weight, weight])

    edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
    edge_weights = torch.tensor(edge_weights, dtype=torch.float)

    print(f"[INFO] Graph edges: {edge_index.shape[1]} edges (bidirectional)")

    # Scale features
    print("[INFO] Scaling features with RobustScaler...")
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X)

    # Convert to PyTorch tensors
    x = torch.tensor(X_scaled, dtype=torch.float)
    y = torch.tensor(y, dtype=torch.long)

    print(f"[INFO] Final graph structure:")
    print(f"  - Nodes: {x.shape[0]}")
    print(f"  - Node features: {x.shape[1]}")
    f"  - Edges: {edge_index.shape[1]}"
    print(f"  - Classes: Licit={sum(y==0)}, Illicit={sum(y==1)}")

    # Create PyTorch Geometric data object
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_weights, y=y)

    # Save checkpoint
    save_checkpoint('graph_built', {
        'data': data,
        'addr_to_idx': addr_to_idx,
        'feature_cols': feature_cols,
        'scaler': scaler,
        'X_scaled': X_scaled
    }, "Graph construction checkpoint saved")

# STEP 5: Model Definition (Updated for batch processing and torch.nn.BatchNorm1d)
class ImprovedGNN(nn.Module):
    def __init__(self, num_features, hidden_dim=128, num_classes=2, dropout=0.3):
        super(ImprovedGNN, self).__init__()

        # Graph attention layers for better feature learning
        self.gat1 = GATConv(num_features, hidden_dim, heads=4, dropout=dropout)
        # Use standard torch.nn.BatchNorm1d
        self.bn1 = torch.nn.BatchNorm1d(hidden_dim * 4)

        self.gat2 = GATConv(hidden_dim * 4, hidden_dim, heads=4, dropout=dropout)
        # Use standard torch.nn.BatchNorm1d
        self.bn2 = torch.nn.BatchNorm1d(hidden_dim * 4)

        self.gat3 = GATConv(hidden_dim * 4, hidden_dim // 2, heads=2, dropout=dropout)
        # Use standard torch.nn.BatchNorm1d
        self.bn3 = torch.nn.BatchNorm1d(hidden_dim) # output shape after gat3 is hidden_dim // 2 * 2 = hidden_dim

        # Feature-only path (like XGBoost)
        self.feature_path = nn.Sequential(
            nn.Linear(num_features, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.Dropout(dropout)
        )

        # Combined classifier
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim + hidden_dim // 2, hidden_dim // 2),  # Graph + Feature paths
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 32),
            nn.ReLU(),
            nn.Dropout(dropout // 2),
            nn.Linear(32, num_classes)
        )

        self.dropout = dropout

    def forward(self, x, edge_index, batch=None):
        # Graph path
        h1 = F.elu(self.gat1(x, edge_index))
        h1 = self.bn1(h1)
        h1 = F.dropout(h1, p=self.dropout, training=self.training)

        h2 = F.elu(self.gat2(h1, edge_index))
        h2 = self.bn2(h2)
        h2 = F.dropout(h2, p=self.dropout, training=self.training)

        h3 = F.elu(self.gat3(h2, edge_index))
        h3 = self.bn3(h3)
        graph_features = F.dropout(h3, p=self.dropout, training=self.training)

        # Feature-only path (mimics XGBoost)
        feature_only = self.feature_path(x)

        # Combine both paths
        combined = torch.cat([graph_features, feature_only], dim=1)

        # Classification
        out = self.classifier(combined)

        return out

# STEP 6: Training setup with NeighborLoader - FIXED CUDA DETECTION
def save_training_checkpoint(epoch, model, optimizer, scheduler, best_f1, patience_counter, train_losses, test_f1_scores):
    """Save training checkpoint"""
    training_state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
        'best_f1': best_f1,
        'patience_counter': patience_counter,
        'train_losses': train_losses,
        'test_f1_scores': test_f1_scores
    }
    torch.save(training_state, os.path.join(CHECKPOINT_DIR, 'training_checkpoint.pth'))
    print(f"💾 Training checkpoint saved at epoch {epoch}")

# FORCE CUDA DETECTION - Don't load device from checkpoint
print("[INFO] Forcing CUDA device detection...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"[INFO] CUDA Available: {torch.cuda.is_available()}")
print(f"[INFO] Device: {device}")

if torch.cuda.is_available():
    print(f"[INFO] CUDA Device: {torch.cuda.get_device_name(0)}")
    print(f"[INFO] CUDA Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Check for training setup checkpoint - but don't load device from it
if checkpoint_exists('training_setup'):
    print("[RESUME] Loading training setup from checkpoint...")
    checkpoint_data = load_checkpoint('training_setup')
    train_mask = checkpoint_data['train_mask']
    test_mask = checkpoint_data['test_mask']
    # DON'T LOAD DEVICE FROM CHECKPOINT - use current detection
    print(f"[SUCCESS] Resumed training setup")
    print(f"[INFO] Train samples: {train_mask.sum()}")
    print(f"[INFO] Test samples: {test_mask.sum()}")
    print(f"[INFO] Device: {device}")  # Use current device
else:
    print("[INFO] Creating stratified train-test split...")
    train_idx, test_idx = train_test_split(
        range(len(data.y)), test_size=0.2, random_state=42,
        stratify=data.y.numpy()
    )

    train_mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    test_mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    train_mask[train_idx] = True
    test_mask[test_idx] = True

    print(f"[INFO] Train samples: {train_mask.sum()}")
    print(f"[INFO] Test samples: {test_mask.sum()}")
    print(f"[INFO] Device: {device}")

    # Save checkpoint - but device will be re-detected next time
    save_checkpoint('training_setup', {
        'train_mask': train_mask,
        'test_mask': test_mask,
        'device': device  # This will be ignored on reload
    }, "Training setup checkpoint saved")

# Initialize model and training components
print(f"[INFO] Initializing model on device: {device}")
model = ImprovedGNN(num_features=data.x.shape[1], hidden_dim=128, dropout=0.3).to(device)

# IMPORTANT: Move data to device for NeighborLoader
print("[INFO] Moving graph data to device...")
data = data.to(device)

# Weighted loss for imbalanced classes
class_weights_tensor = torch.tensor([class_weight_dict[0], class_weight_dict[1]], dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=20)

# STEP 7: CREATE NEIGHBORLOADER FOR MEMORY-EFFICIENT TRAINING
print("[INFO] Creating NeighborLoader for memory-efficient training...")

# Create training and test loaders with neighbor sampling
train_loader = NeighborLoader(
    data,
    num_neighbors=[15, 10, 5],  # Sample 15 neighbors for layer 1, 10 for layer 2, 5 for layer 3
    batch_size=1024,  # Process 1024 nodes at a time
    input_nodes=train_mask,
    shuffle=True,
    num_workers=0  # Set to 0 for Colab compatibility
)

test_loader = NeighborLoader(
    data,
    num_neighbors=[15, 10, 5],
    batch_size=1024,
    input_nodes=test_mask,
    shuffle=False,
    num_workers=0
)

print(f"[INFO] Training batches: {len(train_loader)}")
print(f"[INFO] Test batches: {len(test_loader)}")

# Check for training resume
training_checkpoint_path = os.path.join(CHECKPOINT_DIR, 'training_checkpoint.pth')
start_epoch = 0
best_f1 = 0
patience_counter = 0
train_losses = []
test_f1_scores = []

if os.path.exists(training_checkpoint_path):
    print("[RESUME] Loading training checkpoint...")
    checkpoint = torch.load(training_checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if checkpoint['scheduler_state_dict']:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    best_f1 = checkpoint['best_f1']
    patience_counter = checkpoint['patience_counter']
    train_losses = checkpoint['train_losses']
    test_f1_scores = checkpoint['test_f1_scores']
    print(f"[SUCCESS] Resumed training from epoch {start_epoch}, best F1: {best_f1:.4f}")

print(f"[INFO] Model parameters: {sum(p.numel() for p in model.parameters())}")
print(f"[INFO] Model device: {next(model.parameters()).device}")
print(f"[INFO] Data device: {data.x.device}")

# STEP 8: Training loop with NeighborLoader (MEMORY EFFICIENT)
print("\n" + "="*70)
print("🚀 TRAINING WITH NEIGHBORLOADER - MEMORY EFFICIENT ON CUDA")
print("="*70)

patience = 50
max_epochs = 500

for epoch in range(start_epoch, max_epochs):
    # Training with mini-batches
    model.train()
    total_loss = 0
    num_batches = 0

    for batch in train_loader:
        # Batch should already be on the correct device since data is on device
        # But let's be explicit
        batch = batch.to(device)
        optimizer.zero_grad()

        # Forward pass on batch
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        num_batches += 1

    avg_loss = total_loss / num_batches
    train_losses.append(avg_loss)

    # Evaluation every 10 epochs
    if epoch % 10 == 0:
        model.eval()
        all_preds = []
        all_labels = []
        all_probs = []

        with torch.no_grad():
            for batch in test_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index)
                pred_proba = F.softmax(out, dim=1)[:, 1]
                pred = out.argmax(dim=1)

                all_preds.extend(pred.cpu().numpy())
                all_labels.extend(batch.y.cpu().numpy())
                all_probs.extend(pred_proba.cpu().numpy())

        # Convert to numpy arrays
        y_test_np = np.array(all_labels)
        y_pred_np = np.array(all_preds)
        y_pred_proba_np = np.array(all_probs)

        # Find optimal threshold
        if len(np.unique(y_test_np)) > 1:
            precision, recall, thresholds = precision_recall_curve(y_test_np, y_pred_proba_np)
            f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
            best_threshold_idx = np.argmax(f1_scores)
            best_threshold = thresholds[best_threshold_idx] if len(thresholds) > 0 else 0.5

            y_pred_threshold = (y_pred_proba_np >= best_threshold).astype(int)
            test_f1 = f1_score(y_test_np, y_pred_threshold)
        else:
            test_f1 = 0
            best_threshold = 0.5

        test_f1_scores.append(test_f1)

        # Calculate training accuracy (sample from training loader)
        model.eval()
        train_preds_agg = []
        train_labels_agg = []
        train_batches_checked = 0

        with torch.no_grad():
            for batch in train_loader:
                if train_batches_checked >= 5:  # Sample first 5 batches for speed
                    break
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index)
                pred = out.argmax(dim=1)
                train_preds_agg.extend(pred.cpu().numpy())
                train_labels_agg.extend(batch.y.cpu().numpy())
                train_batches_checked += 1

        train_acc = np.mean(np.array(train_preds_agg) == np.array(train_labels_agg))
        test_acc = np.mean(y_pred_np == y_test_np)

        print(f'Epoch {epoch:03d}, Loss: {avg_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, F1: {test_f1:.4f}, Threshold: {best_threshold:.3f}')

        # Early stopping based on F1 score
        if test_f1 > best_f1:
            best_f1 = test_f1
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, 'best_gnn_model.pth'))
        else:
            patience_counter += 1

        # Save training checkpoint every 20 epochs
        if epoch % 20 == 0:
            save_training_checkpoint(epoch, model, optimizer, scheduler, best_f1, patience_counter, train_losses, test_f1_scores)

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

        scheduler.step(test_f1)

print(f"\n✅ TRAINING COMPLETED ON: {device}")
print(f"📊 Final device check - Model: {next(model.parameters()).device}")

# Final checkpoint save
save_training_checkpoint(epoch, model, optimizer, scheduler, best_f1, patience_counter, train_losses, test_f1_scores)

# Load best model for final evaluation
best_model_path = os.path.join(CHECKPOINT_DIR, 'best_gnn_model.pth')
if os.path.exists(best_model_path):
    model.load_state_dict(torch.load(best_model_path, map_location=device))

# STEP 9: Final evaluation with NeighborLoader (using aggregated results from test loader)
print("\n" + "="*70)
print("📊 FINAL EVALUATION WITH NEIGHBORLOADER (Aggregated Test Batches)")
print("="*70)

model.eval()
all_preds = []
all_labels = []
all_probs = []

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        out = model(batch.x, batch.edge_index)
        pred_proba = F.softmax(out, dim=1)[:, 1]
        pred = out.argmax(dim=1)

        all_preds.extend(pred.cpu().numpy())
        all_labels.extend(batch.y.cpu().numpy())
        all_probs.extend(pred_proba.cpu().numpy())

# Convert to numpy arrays
y_test = np.array(all_labels)
y_pred_proba_test = np.array(all_probs)

# Find optimal threshold
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba_test)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
best_threshold_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[best_threshold_idx] if len(thresholds) > 0 else 0.5 # Handle case with no positive predictions

y_pred_test = (y_pred_proba_test >= optimal_threshold).astype(int)

# Calculate metrics
# Note: Accuracy is calculated on aggregated batch predictions, not the true overall test set.
test_acc = np.mean(np.array(all_preds) == y_test)
# AUC and F1 are correctly calculated on aggregated test predictions and probabilities
auc_score = roc_auc_score(y_test, y_pred_proba_test) if len(np.unique(y_test)) > 1 else 0.0 # Handle case with no positive labels in test set
final_f1 = f1_score(y_test, y_pred_test) if len(np.unique(y_test)) > 1 else 0.0


print(f"Test Accuracy (Aggregated): {test_acc:.4f}")
print(f"AUC Score (Aggregated): {auc_score:.4f}")
print(f"F1 Score (Aggregated): {final_f1:.4f}")
print(f"Optimal Threshold: {optimal_threshold:.3f}")

print("\nClassification Report (Aggregated Test Batches):")
# Ensure target names match unique labels present in y_test
target_names = ['Licit', 'Illicit']
report = classification_report(y_test, y_pred_test, target_names=target_names, zero_division=0)
print(report)


print("\nConfusion Matrix (Aggregated Test Batches):")
cm = confusion_matrix(y_test, y_pred_test)
print("   Predicted:  Licit  Illicit")
print(f"   Licit:      {cm[0,0]:5d}     {cm[0,1]:4d}")
print(f"   Illicit:      {cm[1,0]:3d}     {cm[1,1]:4d}")


# STEP 10: Save the final model
print("\n" + "="*70)
print("💾 SAVING FINAL MODEL WITH NEIGHBORLOADER SUPPORT")
print("="*70)

model_data = {
    'model_state_dict': model.state_dict(),
    'scaler': scaler,
    'addr_to_idx': addr_to_idx,
    'feature_cols': feature_cols,
    'optimal_threshold': optimal_threshold,
    'class_weights': class_weight_dict,
    'model_config': {
        'num_features': data.x.shape[1],
        'hidden_dim': 128,
        'num_classes': 2,
        'dropout': 0.3
    },
    'training_history': {
        'train_losses': train_losses,
        'test_f1_scores': test_f1_scores,
        'final_metrics': {
            'test_accuracy': test_acc, # Note: Aggregated accuracy
            'auc_score': auc_score,
            'f1_score': final_f1,
            'optimal_threshold': optimal_threshold
        }
    }
}

torch.save(model_data, 'improved_gnn_ransomware_model_neighborloader.pth')
print("[SUCCESS] Final NeighborLoader model saved!")

# Clean up checkpoints (optional)
print("\n🧹 CLEANUP OPTIONS:")
print("To clean up checkpoints and save space, run:")
print("  import shutil; shutil.rmtree('/content/checkpoints')")

print(f"\n🎯 NEIGHBORLOADER TRAINING ADOPTED:")
print(f"  ✅ Re-enabled NeighborLoader for memory efficiency")
print(f"  ✅ Training and evaluation using mini-batches")
print(f"  ✅ Requires torch-sparse (installed)")
print(f"  ⚠️ Evaluation metrics are aggregated over test batches, not the entire test set at once.")

print(f"\n📊 Training Attempted:")
print(f"  - Training with NeighborLoader should now proceed if torch-sparse is correctly installed.")

C:\Users\CATURWARGA COMPUTER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch_geometric\__init__.py:4: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  import torch_geometric.typing
C:\Users\CATURWARGA COMPUTER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 IMPROVED GNN RANSOMWARE DETECTION MODEL WITH NEIGHBORLOADER
[RESUME] Loading data from checkpoint...
[SUCCESS] Resumed with 1268260 wallet records and 2868964 edges
[RESUME] Loading feature engineering from checkpoint...
[SUCCESS] Resumed with 68 columns
[RESUME] Loading cleaned data from checkpoint...
[SUCCESS] Resumed with 367472 cleaned addresses
[INFO] Class distribution: {0: 338871, 1: 28601}
[INFO] Class weights: {0: np.float64(0.5422004243502689), 1: np.float64(6.424111045068354)}
[RESUME] Loading graph from checkpoint...
[SUCCESS] Resumed with graph: 265354 nodes, 2236444 edges
[INFO] Forcing CUDA device detection...
[INFO] CUDA Available: True
[INFO] Device: cuda
[INFO] CUDA Device: NVIDIA GeForce RTX 2060 with Max-Q Design
[INFO] CUDA Memory: 6.44 GB
[RESUME] Loading training setup from checkpoint...
[SUCCESS] Resumed training setup
[INFO] Train samples: 212283
[INFO] Test samples: 53071
[INFO] Device: cuda
[INFO] Initializing model on device: cuda
[INFO] Moving graph data 

KeyboardInterrupt: 

In [1]:
# Quick sanity check
print("Data sanity check:")
print(f"Training labels: {data.y[train_mask].unique()}")
print(f"Test labels: {data.y[test_mask].unique()}")
print(f"Training class distribution: {torch.bincount(data.y[train_mask])}")
print(f"Test class distribution: {torch.bincount(data.y[test_mask])}")

Data sanity check:


NameError: name 'data' is not defined

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv, global_mean_pool
# Re-import NeighborLoader for mini-batch training
from torch_geometric.loader import NeighborLoader
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, f1_score
from sklearn.utils.class_weight import compute_class_weight
import warnings
import pickle
import os
import json
warnings.filterwarnings('ignore')


# CHECKPOINT SYSTEM
CHECKPOINT_DIR = 'Ransomware_Checkpoints'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

def save_checkpoint(checkpoint_name, data, message="Checkpoint saved"):
    """Save checkpoint data"""
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f"{checkpoint_name}.pkl")
    with open(checkpoint_path, 'wb') as f:
        pickle.dump(data, f)
    print(f"✅ {message}: {checkpoint_path}")

def load_checkpoint(checkpoint_name):
    """Load checkpoint data"""
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f"{checkpoint_name}.pkl")
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'rb') as f:
            return pickle.load(f)
    return None

def checkpoint_exists(checkpoint_name):
    """Check if checkpoint exists"""
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f"{checkpoint_name}.pkl")
    return os.path.exists(checkpoint_path)

# Updated title for clarity
print("🚀 IMPROVED GNN RANSOMWARE DETECTION MODEL WITH NEIGHBORLOADER")
print("="*70)

# STEP 1: Check for data loading checkpoint
if checkpoint_exists('data_loaded'):
    print("[RESUME] Loading data from checkpoint...")
    checkpoint_data = load_checkpoint('data_loaded')
    wallets_df = checkpoint_data['wallets_df']
    edges_df = checkpoint_data['edges_df']
    print(f"[SUCCESS] Resumed with {len(wallets_df)} wallet records and {len(edges_df)} edges")
else:
    print("[INFO] Loading wallet features and classes...")
    wallets_df = pd.read_csv('Elliptic++ Dataset/wallets_features_classes_combined.csv')
    print(f"[SUCCESS] Loaded {len(wallets_df)} wallet records")

    print("[INFO] Loading address-to-address edges...")
    edges_df = pd.read_csv('Elliptic++ Dataset/AddrAddr_edgelist.csv')
    print(f"[SUCCESS] Loaded {len(edges_df)} edges")

    # Save checkpoint
    save_checkpoint('data_loaded', {
        'wallets_df': wallets_df,
        'edges_df': edges_df
    }, "Data loading checkpoint saved")

# STEP 2: Check for feature engineering checkpoint
def create_enhanced_pattern_features(df):
    """Create the EXACT same enhanced features as XGBoost model"""

    # Fill missing values first (same as XGBoost)
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df[numeric_columns] = df[numeric_columns].fillna(0)

    # 1. Partner Transaction Ratio (connectivity)
    df['partner_transaction_ratio'] = (
        df.get('transacted_w_address_total', 0) /
        (df.get('total_txs', 1) + 1e-8)
    )

    # 2. Activity Density (txs per block)
    df['activity_density'] = (
        df.get('total_txs', 0) /
        (df.get('lifetime_in_blocks', 1) + 1e-8)
    )

    # 3. Transaction Size Variance (volatility)
    df['transaction_size_variance'] = (
        df.get('btc_transacted_max', 0) - df.get('btc_transacted_min', 0)
    ) / (df.get('btc_transacted_mean', 1) + 1e-8)

    # 4. Flow Imbalance (money laundering indicator)
    df['flow_imbalance'] = (
        (df.get('btc_sent_total', 0) - df.get('btc_received_total', 0)) /
        (df.get('btc_transacted_total', 1) + 1e-8)
    )

    # 5. Temporal Spread (time pattern)
    df['temporal_spread'] = (
        df.get('last_block_appeared_in', 0) - df.get('first_block_appeared_in', 0)
    ) / (df.get('num_timesteps_appeared_in', 1) + 1e-8)

    # 6. Fee Percentile (urgency indicator)
    df['fee_percentile'] = (
        df.get('fees_total', 0) /
        (df.get('btc_transacted_total', 1) + 1e-8)
    )

    # 7. Interaction Intensity (network centrality)
    df['interaction_intensity'] = (
        df.get('num_addr_transacted_multiple', 0) /
        (df.get('transacted_w_address_total', 1) + 1e-8)
    )

    # 8. Value Per Transaction (transaction size)
    df['value_per_transaction'] = (
        df.get('btc_transacted_total', 0) /
        (df.get('total_txs', 1) + 1e-8)
    )

    # 9. RANSOMWARE-SPECIFIC: Burst Activity (rapid txs)
    df['burst_activity'] = (
        df.get('total_txs', 0) * df.get('activity_density', 0)
    )

    # 10. RANSOMWARE-SPECIFIC: Mixing Intensity (obfuscation)
    df['mixing_intensity'] = (
        df.get('partner_transaction_ratio', 0) * df.get('interaction_intensity', 0)
    )

    return df

if checkpoint_exists('features_engineered'):
    print("[RESUME] Loading feature engineering from checkpoint...")
    checkpoint_data = load_checkpoint('features_engineered')
    wallets_df = checkpoint_data['wallets_df']
    print(f"[SUCCESS] Resumed with {len(wallets_df.columns)} columns")
else:
    print("[INFO] Creating enhanced features...")
    wallets_df = create_enhanced_pattern_features(wallets_df)
    print(f"[SUCCESS] Enhanced features created. Now have {len(wallets_df.columns)} columns")

    # Save checkpoint
    save_checkpoint('features_engineered', {
        'wallets_df': wallets_df
    }, "Feature engineering checkpoint saved")

# STEP 3: Check for data cleaning checkpoint
if checkpoint_exists('data_cleaned'):
    print("[RESUME] Loading cleaned data from checkpoint...")
    checkpoint_data = load_checkpoint('data_cleaned')
    wallets_clean = checkpoint_data['wallets_clean']
    class_weight_dict = checkpoint_data['class_weight_dict']
    print(f"[SUCCESS] Resumed with {len(wallets_clean)} cleaned addresses")
    print(f"[INFO] Class distribution: {wallets_clean['class'].value_counts().to_dict()}")
    print(f"[INFO] Class weights: {class_weight_dict}")
else:
    print("[INFO] Cleaning data with same logic as XGBoost...")

    # Keep only labeled addresses (1=Illicit, 2=Licit)
    wallets_clean = wallets_df[wallets_df['class'].isin([1, 2])].copy()
    print(f"[INFO] Labeled addresses: {len(wallets_clean)}")

    # Remap classes: 1->1 (Illicit), 2->0 (Licit) - SAME AS XGBOOST
    wallets_clean['class'] = wallets_clean['class'].map({1: 1, 2: 0})
    print(f"[INFO] Class distribution: {wallets_clean['class'].value_counts().to_dict()}")

    # Calculate class weights (same as XGBoost)
    class_weights = compute_class_weight('balanced', classes=np.unique(wallets_clean['class']), y=wallets_clean['class'])
    class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
    print(f"[INFO] Class weights: {class_weight_dict}")

    # Save checkpoint
    save_checkpoint('data_cleaned', {
        'wallets_clean': wallets_clean,
        'class_weight_dict': class_weight_dict
    }, "Data cleaning checkpoint saved")

# STEP 4: Check for graph construction checkpoint
if checkpoint_exists('graph_built'):
    print("[RESUME] Loading graph from checkpoint...")
    checkpoint_data = load_checkpoint('graph_built')
    data = checkpoint_data['data']
    addr_to_idx = checkpoint_data['addr_to_idx']
    feature_cols = checkpoint_data['feature_cols']
    scaler = checkpoint_data['scaler']
    X_scaled = checkpoint_data['X_scaled']
    print(f"[SUCCESS] Resumed with graph: {data.x.shape[0]} nodes, {data.edge_index.shape[1]} edges")
else:
    print("[INFO] Building improved graph structure...")
    print("This is the slow part - creating address mappings and features...")

    # Create address to index mapping
    unique_addresses = wallets_clean['address'].unique()
    addr_to_idx = {addr: idx for idx, addr in enumerate(unique_addresses)}

    # Prepare features and labels
    exclude_cols = ['address', 'Time step', 'class']
    feature_cols = [col for col in wallets_clean.columns if col not in exclude_cols]

    features_list = []
    labels_list = []
    address_list = []

    print(f"[INFO] Processing {len(unique_addresses)} unique addresses...")
    for i, addr in enumerate(unique_addresses):
        if i % 10000 == 0:
            print(f"  Progress: {i}/{len(unique_addresses)} ({i/len(unique_addresses)*100:.1f}%)")

        addr_data = wallets_clean[wallets_clean['address'] == addr].iloc[0]
        features_list.append(addr_data[feature_cols].values)
        labels_list.append(addr_data['class'])
        address_list.append(addr)

    X = np.array(features_list)
    y = np.array(labels_list)

    print(f"[INFO] Feature matrix shape: {X.shape}")
    print(f"[INFO] Labels shape: {y.shape}")

    # Create edge list with weights
    print("[INFO] Creating weighted edges...")
    edge_list = []
    edge_weights = []

    print(f"[INFO] Processing {len(edges_df)} edges...")
    for i, (_, row) in enumerate(edges_df.iterrows()):
        if i % 50000 == 0:
            print(f"  Edge progress: {i}/{len(edges_df)} ({i/len(edges_df)*100:.1f}%)")

        input_addr = row['input_address']
        output_addr = row['output_address']

        if input_addr in addr_to_idx and output_addr in addr_to_idx:
            input_idx = addr_to_idx[input_addr]
            output_idx = addr_to_idx[output_addr]

            # Add both directions for undirected graph
            edge_list.extend([[input_idx, output_idx], [output_idx, input_idx]])

            # Use transaction amount as edge weight if available
            weight = row.get('value', 1.0) if 'value' in row else 1.0
            edge_weights.extend([weight, weight])

    edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
    edge_weights = torch.tensor(edge_weights, dtype=torch.float)

    print(f"[INFO] Graph edges: {edge_index.shape[1]} edges (bidirectional)")

    # Scale features
    print("[INFO] Scaling features with RobustScaler...")
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X)

    # Convert to PyTorch tensors
    x = torch.tensor(X_scaled, dtype=torch.float)
    y = torch.tensor(y, dtype=torch.long)

    print(f"[INFO] Final graph structure:")
    print(f"  - Nodes: {x.shape[0]}")
    print(f"  - Node features: {x.shape[1]}")
    f"  - Edges: {edge_index.shape[1]}"
    print(f"  - Classes: Licit={sum(y==0)}, Illicit={sum(y==1)}")

    # Create PyTorch Geometric data object
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_weights, y=y)

    # Save checkpoint
    save_checkpoint('graph_built', {
        'data': data,
        'addr_to_idx': addr_to_idx,
        'feature_cols': feature_cols,
        'scaler': scaler,
        'X_scaled': X_scaled
    }, "Graph construction checkpoint saved")

# STEP 5: Model Definition (Updated for batch processing and torch.nn.BatchNorm1d)
class EnhancedGNN(nn.Module):
    def __init__(self, num_features, hidden_dim=128, num_classes=2, dropout=0.3):
        super(EnhancedGNN, self).__init__()
        
        self.num_features = num_features
        self.hidden_dim = hidden_dim
        
        # Graph attention layers with residual connections
        self.gat1 = GATConv(num_features, hidden_dim, heads=4, dropout=dropout)
        self.bn1 = torch.nn.BatchNorm1d(hidden_dim * 4)
        
        # Residual connection layer
        self.residual1 = nn.Linear(num_features, hidden_dim * 4)
        
        self.gat2 = GATConv(hidden_dim * 4, hidden_dim, heads=4, dropout=dropout)
        self.bn2 = torch.nn.BatchNorm1d(hidden_dim * 4)
        
        # Residual connection layer
        self.residual2 = nn.Linear(hidden_dim * 4, hidden_dim * 4)
        
        self.gat3 = GATConv(hidden_dim * 4, hidden_dim // 2, heads=2, dropout=dropout)
        self.bn3 = torch.nn.BatchNorm1d(hidden_dim)
        
        # Enhanced feature path with attention
        self.feature_path = nn.Sequential(
            nn.Linear(num_features, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.Dropout(dropout)
        )
        
        # Attention fusion for combining paths
        self.attention_fusion = nn.Sequential(
            nn.Linear(hidden_dim + hidden_dim // 2, hidden_dim // 4),
            nn.Tanh(),
            nn.Linear(hidden_dim // 4, 2),  # 2 attention weights for 2 paths
            nn.Softmax(dim=1)
        )
        
        # Enhanced classifier
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim + hidden_dim // 2, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 64),
            nn.ReLU(),
            nn.Dropout(dropout // 2),
            nn.Linear(64, num_classes)
        )
        
        self.dropout = dropout
    
    def forward(self, x, edge_index, batch=None):
        # Graph path with residual connections
        h1 = F.elu(self.gat1(x, edge_index))
        h1 = self.bn1(h1)
        h1 = h1 + self.residual1(x)  # Residual connection
        h1 = F.dropout(h1, p=self.dropout, training=self.training)
        
        h2 = F.elu(self.gat2(h1, edge_index))
        h2 = self.bn2(h2)
        h2 = h2 + self.residual2(h1)  # Residual connection
        h2 = F.dropout(h2, p=self.dropout, training=self.training)
        
        h3 = F.elu(self.gat3(h2, edge_index))
        h3 = self.bn3(h3)
        graph_features = F.dropout(h3, p=self.dropout, training=self.training)
        
        # Feature-only path
        feature_only = self.feature_path(x)
        
        # Attention-based fusion
        combined = torch.cat([graph_features, feature_only], dim=1)
        attention_weights = self.attention_fusion(combined)
        
        # Apply attention weights
        weighted_graph = graph_features * attention_weights[:, 0:1]
        weighted_features = feature_only * attention_weights[:, 1:2]
        
        # Final combination
        final_features = torch.cat([weighted_graph, weighted_features], dim=1)
        
        # Classification
        out = self.classifier(final_features)
        return out

# STEP 6: Training setup with NeighborLoader - FIXED CUDA DETECTION
def save_training_checkpoint(epoch, model, optimizer, scheduler, best_f1, patience_counter, train_losses, test_f1_scores):
    """Save training checkpoint"""
    training_state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
        'best_f1': best_f1,
        'patience_counter': patience_counter,
        'train_losses': train_losses,
        'test_f1_scores': test_f1_scores
    }
    torch.save(training_state, os.path.join(CHECKPOINT_DIR, 'training_checkpoint.pth'))
    print(f"💾 Training checkpoint saved at epoch {epoch}")

# FORCE CUDA DETECTION - Don't load device from checkpoint
print("[INFO] Forcing CUDA device detection...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"[INFO] CUDA Available: {torch.cuda.is_available()}")
print(f"[INFO] Device: {device}")

if torch.cuda.is_available():
    print(f"[INFO] CUDA Device: {torch.cuda.get_device_name(0)}")
    print(f"[INFO] CUDA Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Check for training setup checkpoint - but don't load device from it
if checkpoint_exists('training_setup'):
    print("[RESUME] Loading training setup from checkpoint...")
    checkpoint_data = load_checkpoint('training_setup')
    train_mask = checkpoint_data['train_mask']
    test_mask = checkpoint_data['test_mask']
    # DON'T LOAD DEVICE FROM CHECKPOINT - use current detection
    print(f"[SUCCESS] Resumed training setup")
    print(f"[INFO] Train samples: {train_mask.sum()}")
    print(f"[INFO] Test samples: {test_mask.sum()}")
    print(f"[INFO] Device: {device}")  # Use current device
else:
    print("[INFO] Creating stratified train-test split...")
    train_idx, test_idx = train_test_split(
        range(len(data.y)), test_size=0.2, random_state=42,
        stratify=data.y.numpy()
    )

    train_mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    test_mask = torch.zeros(data.y.size(0), dtype=torch.bool)
    train_mask[train_idx] = True
    test_mask[test_idx] = True

    print(f"[INFO] Train samples: {train_mask.sum()}")
    print(f"[INFO] Test samples: {test_mask.sum()}")
    print(f"[INFO] Device: {device}")

    # Save checkpoint - but device will be re-detected next time
    save_checkpoint('training_setup', {
        'train_mask': train_mask,
        'test_mask': test_mask,
        'device': device  # This will be ignored on reload
    }, "Training setup checkpoint saved")

# Initialize model and training components
print(f"[INFO] Initializing model on device: {device}")
model = ImprovedGNN(num_features=data.x.shape[1], hidden_dim=128, dropout=0.3).to(device)

# IMPORTANT: Move data to device for NeighborLoader
print("[INFO] Moving graph data to device...")
data = data.to(device)

# Weighted loss for imbalanced classes
class_weights_tensor = torch.tensor([class_weight_dict[0], class_weight_dict[1]], dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=20)

# STEP 7: CREATE NEIGHBORLOADER FOR MEMORY-EFFICIENT TRAINING
print("[INFO] Creating NeighborLoader for memory-efficient training...")

# Create training and test loaders with neighbor sampling
train_loader = NeighborLoader(
    data,
    num_neighbors=[15, 10, 5],  # Sample 15 neighbors for layer 1, 10 for layer 2, 5 for layer 3
    batch_size=1024,  # Process 1024 nodes at a time
    input_nodes=train_mask,
    shuffle=True,
    num_workers=0  # Set to 0 for Colab compatibility
)

test_loader = NeighborLoader(
    data,
    num_neighbors=[15, 10, 5],
    batch_size=1024,
    input_nodes=test_mask,
    shuffle=False,
    num_workers=0
)

print(f"[INFO] Training batches: {len(train_loader)}")
print(f"[INFO] Test batches: {len(test_loader)}")

# Check for training resume
training_checkpoint_path = os.path.join(CHECKPOINT_DIR, 'training_checkpoint.pth')
start_epoch = 0
best_f1 = 0
patience_counter = 0
train_losses = []
test_f1_scores = []

if os.path.exists(training_checkpoint_path):
    print("[RESUME] Loading training checkpoint...")
    checkpoint = torch.load(training_checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if checkpoint['scheduler_state_dict']:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    best_f1 = checkpoint['best_f1']
    patience_counter = checkpoint['patience_counter']
    train_losses = checkpoint['train_losses']
    test_f1_scores = checkpoint['test_f1_scores']
    print(f"[SUCCESS] Resumed training from epoch {start_epoch}, best F1: {best_f1:.4f}")

print(f"[INFO] Model parameters: {sum(p.numel() for p in model.parameters())}")
print(f"[INFO] Model device: {next(model.parameters()).device}")
print(f"[INFO] Data device: {data.x.device}")

# STEP 8: Training loop with NeighborLoader (MEMORY EFFICIENT)
print("\n" + "="*70)
print("🚀 TRAINING WITH NEIGHBORLOADER - MEMORY EFFICIENT ON CUDA")
print("="*70)

patience = 50
max_epochs = 500

for epoch in range(start_epoch, max_epochs):
    # Training with mini-batches
    model.train()
    total_loss = 0
    num_batches = 0

    for batch in train_loader:
        # Batch should already be on the correct device since data is on device
        # But let's be explicit
        batch = batch.to(device)
        optimizer.zero_grad()

        # Forward pass on batch
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        num_batches += 1

    avg_loss = total_loss / num_batches
    train_losses.append(avg_loss)

    # Evaluation every 10 epochs
    if epoch % 10 == 0:
        model.eval()
        all_preds = []
        all_labels = []
        all_probs = []

        with torch.no_grad():
            for batch in test_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index)
                pred_proba = F.softmax(out, dim=1)[:, 1]
                pred = out.argmax(dim=1)

                all_preds.extend(pred.cpu().numpy())
                all_labels.extend(batch.y.cpu().numpy())
                all_probs.extend(pred_proba.cpu().numpy())

        # Convert to numpy arrays
        y_test_np = np.array(all_labels)
        y_pred_np = np.array(all_preds)
        y_pred_proba_np = np.array(all_probs)

        # Find optimal threshold
        if len(np.unique(y_test_np)) > 1:
            precision, recall, thresholds = precision_recall_curve(y_test_np, y_pred_proba_np)
            f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
            best_threshold_idx = np.argmax(f1_scores)
            best_threshold = thresholds[best_threshold_idx] if len(thresholds) > 0 else 0.5

            y_pred_threshold = (y_pred_proba_np >= best_threshold).astype(int)
            test_f1 = f1_score(y_test_np, y_pred_threshold)
        else:
            test_f1 = 0
            best_threshold = 0.5

        test_f1_scores.append(test_f1)

        # Calculate training accuracy (sample from training loader)
        model.eval()
        train_preds_agg = []
        train_labels_agg = []
        train_batches_checked = 0

        with torch.no_grad():
            for batch in train_loader:
                if train_batches_checked >= 5:  # Sample first 5 batches for speed
                    break
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index)
                pred = out.argmax(dim=1)
                train_preds_agg.extend(pred.cpu().numpy())
                train_labels_agg.extend(batch.y.cpu().numpy())
                train_batches_checked += 1

        train_acc = np.mean(np.array(train_preds_agg) == np.array(train_labels_agg))
        test_acc = np.mean(y_pred_np == y_test_np)

        print(f'Epoch {epoch:03d}, Loss: {avg_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, F1: {test_f1:.4f}, Threshold: {best_threshold:.3f}')

        # Early stopping based on F1 score
        if test_f1 > best_f1:
            best_f1 = test_f1
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, 'best_gnn_model.pth'))
        else:
            patience_counter += 1

        # Save training checkpoint every 20 epochs
        if epoch % 20 == 0:
            save_training_checkpoint(epoch, model, optimizer, scheduler, best_f1, patience_counter, train_losses, test_f1_scores)

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

        scheduler.step(test_f1)

print(f"\n✅ TRAINING COMPLETED ON: {device}")
print(f"📊 Final device check - Model: {next(model.parameters()).device}")

# Final checkpoint save
save_training_checkpoint(epoch, model, optimizer, scheduler, best_f1, patience_counter, train_losses, test_f1_scores)

# Load best model for final evaluation
best_model_path = os.path.join(CHECKPOINT_DIR, 'best_gnn_model.pth')
if os.path.exists(best_model_path):
    model.load_state_dict(torch.load(best_model_path, map_location=device))

# STEP 9: Final evaluation with NeighborLoader (using aggregated results from test loader)
print("\n" + "="*70)
print("📊 FINAL EVALUATION WITH NEIGHBORLOADER (Aggregated Test Batches)")
print("="*70)

model.eval()
all_preds = []
all_labels = []
all_probs = []

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        out = model(batch.x, batch.edge_index)
        pred_proba = F.softmax(out, dim=1)[:, 1]
        pred = out.argmax(dim=1)

        all_preds.extend(pred.cpu().numpy())
        all_labels.extend(batch.y.cpu().numpy())
        all_probs.extend(pred_proba.cpu().numpy())

# Convert to numpy arrays
y_test = np.array(all_labels)
y_pred_proba_test = np.array(all_probs)

# Find optimal threshold
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba_test)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
best_threshold_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[best_threshold_idx] if len(thresholds) > 0 else 0.5 # Handle case with no positive predictions

y_pred_test = (y_pred_proba_test >= optimal_threshold).astype(int)

# Calculate metrics
# Note: Accuracy is calculated on aggregated batch predictions, not the true overall test set.
test_acc = np.mean(np.array(all_preds) == y_test)
# AUC and F1 are correctly calculated on aggregated test predictions and probabilities
auc_score = roc_auc_score(y_test, y_pred_proba_test) if len(np.unique(y_test)) > 1 else 0.0 # Handle case with no positive labels in test set
final_f1 = f1_score(y_test, y_pred_test) if len(np.unique(y_test)) > 1 else 0.0


print(f"Test Accuracy (Aggregated): {test_acc:.4f}")
print(f"AUC Score (Aggregated): {auc_score:.4f}")
print(f"F1 Score (Aggregated): {final_f1:.4f}")
print(f"Optimal Threshold: {optimal_threshold:.3f}")

print("\nClassification Report (Aggregated Test Batches):")
# Ensure target names match unique labels present in y_test
target_names = ['Licit', 'Illicit']
report = classification_report(y_test, y_pred_test, target_names=target_names, zero_division=0)
print(report)


print("\nConfusion Matrix (Aggregated Test Batches):")
cm = confusion_matrix(y_test, y_pred_test)
print("   Predicted:  Licit  Illicit")
print(f"   Licit:      {cm[0,0]:5d}     {cm[0,1]:4d}")
print(f"   Illicit:      {cm[1,0]:3d}     {cm[1,1]:4d}")


# STEP 10: Save the final model
print("\n" + "="*70)
print("💾 SAVING FINAL MODEL WITH NEIGHBORLOADER SUPPORT")
print("="*70)

model_data = {
    'model_state_dict': model.state_dict(),
    'scaler': scaler,
    'addr_to_idx': addr_to_idx,
    'feature_cols': feature_cols,
    'optimal_threshold': optimal_threshold,
    'class_weights': class_weight_dict,
    'model_config': {
        'num_features': data.x.shape[1],
        'hidden_dim': 128,
        'num_classes': 2,
        'dropout': 0.3
    },
    'training_history': {
        'train_losses': train_losses,
        'test_f1_scores': test_f1_scores,
        'final_metrics': {
            'test_accuracy': test_acc, # Note: Aggregated accuracy
            'auc_score': auc_score,
            'f1_score': final_f1,
            'optimal_threshold': optimal_threshold
        }
    }
}

torch.save(model_data, 'improved_gnn_ransomware_model_neighborloader.pth')
print("[SUCCESS] Final NeighborLoader model saved!")

# Clean up checkpoints (optional)
print("\n🧹 CLEANUP OPTIONS:")
print("To clean up checkpoints and save space, run:")
print("  import shutil; shutil.rmtree('/content/checkpoints')")

print(f"\n🎯 NEIGHBORLOADER TRAINING ADOPTED:")
print(f"  ✅ Re-enabled NeighborLoader for memory efficiency")
print(f"  ✅ Training and evaluation using mini-batches")
print(f"  ✅ Requires torch-sparse (installed)")
print(f"  ⚠️ Evaluation metrics are aggregated over test batches, not the entire test set at once.")

print(f"\n📊 Training Attempted:")
print(f"  - Training with NeighborLoader should now proceed if torch-sparse is correctly installed.")